# THREDDS Server CONUS Composite Radar 
## Current Month Only

In [1]:
from netCDF4 import Dataset
from netCDF4 import num2date
import glob, os
from datetime import datetime, timedelta
import matplotlib.pyplot as plt

import cartopy.feature as cfeature

import cartopy.crs as ccrs

import numpy as np

import pandas as pd

from matplotlib import patheffects

from siphon.catalog import TDSCatalog

from mpl_toolkits.axes_grid1 import make_axes_locatable
from matplotlib.colors import LinearSegmentedColormap

In [2]:
refl = pd.read_csv("/Users/chowdahead/Documents/refl_cmap.csv")
refl_colors = []
for i in range(0,255):
    refl_colors.append((float(refl["r"][i]),
                               float(refl["g"][i]),
                               float(refl["b"][i]),
                               float(refl["a"][i])
                             ))
cmap_name="refl"
refl_cmap = LinearSegmentedColormap.from_list(
            cmap_name, refl_colors)

## THREDDS Catalog Data Access

In [3]:
top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
ref = top_cat.catalog_refs['Radar Data']
print(ref)
new_cat = ref.follow()
print(new_cat)
model = new_cat.catalog_refs[4]
print(model)
gfs_cat = model.follow()

ds = gfs_cat.datasets[0]

Radar Data
Unidata THREDDS Data Server - Radar Data
Unidata NEXRAD Composite Reflectivity


In [4]:
def query_reflectivity_radar_data(start,minute_delta=0,hour_delta=0,day_delta=0,
                                  west=-122, east=-72, south=23, north=50):
    try:
        top_cat = TDSCatalog('http://thredds.ucar.edu/thredds/catalog.xml')
        ref = top_cat.catalog_refs['Radar Data']

        new_cat = ref.follow()
        model = new_cat.catalog_refs[4]
        gfs_cat = model.follow()

        ds = gfs_cat.datasets[0]
        subset = ds.subset()
        query_data = subset.query()
        query_data.lonlat_box(west=west, east=east, south=south, north=north)
        
        end = start+timedelta(days=day_delta, minutes=minute_delta, hours=hour_delta) 
        
        # Allow for NetCDF files
        query_data.accept('netcdf4')
        
        print(f"query start time:{start}")
        print(f"query end time:{end}")
        query_data.time_range(start,end)
        query_data.variables('Base_reflectivity_surface_layer').add_lonlat()

        # Finally attempt to access the data
        data = subset.get_data(query_data)
        times = data.variables['time']
        lon = data.variables['lon']
        lat = data.variables['lat']
        print("query successful!")
    except:
        print("query not successful...")
    return data,times,lon,lat

In [11]:
def get_radar_vars(data,times,time_index):
    time = data.variables['time'][time_index]
    print(time)
    dtime = num2date(time,times.units)
    
    title_time = dtime.strftime('%Y-%m-%d %H:%MZ')
    print(title_time)
    file_time = dtime.strftime('%Y_%m_%d_%H%MZ')
    #reflec_data = data.variables['Base_reflectivity_surface_layer'][0]
    reflec = data.variables['Base_reflectivity_surface_layer'][time_index]
    #reflec = np.ma.masked_where(reflec<10,reflec)
    
    return title_time,file_time,reflec

In [20]:
def make_text_time_right(ax,end,title_time,
                         color="w",
                       fontsize=8):  
    
    text_time = ax.text(.995, 0.01, 
            title_time,
            horizontalalignment='right', transform=ax.transAxes,
            color=color, fontsize=fontsize, weight='bold',zorder=15)
    outline_effect = [patheffects.withStroke(linewidth=3, foreground='black')]
    text_time.set_path_effects(outline_effect)
    return text_time,ax

def make_text_time_left(ax,
                        color="w",
                       fontsize=8):  
    
    text_time2 = ax.text(0.005, 0.01, 
                "NEXRAD Composite Reflectivity",
                horizontalalignment='left', transform=ax.transAxes,
                color=color, fontsize=fontsize, weight='bold',zorder=15)
    outline_effect = [patheffects.withStroke(linewidth=3, foreground='black')]
    text_time2.set_path_effects(outline_effect)
    return text_time2,ax

In [21]:
def make_map(data,lon,lat,save_path,time_index,Extent=None):  

    title_time,file_time,reflec = get_radar_vars(data,times,time_index=time_index)
    reflec = np.ma.masked_where(reflec<10,reflec)
    
    fig = plt.figure(figsize=(17,11))

    # Add state and country boundaries to plot
    states_boundaries = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_1_states_provinces_lakes',scale='50m', facecolor='none')

    country_borders = cfeature.NaturalEarthFeature(category='cultural',
            name='admin_0_countries',scale='50m', facecolor='none')

    plot_proj = ccrs.LambertConformal(central_latitude=[30, 60], central_longitude=-100)

    ax = plt.subplot(111,projection=ccrs.LambertConformal())
    
    ax.add_feature(states_boundaries,edgecolor='k',linewidth=0.8,linestyle='--',alpha=0.6)
    ax.add_feature(country_borders,edgecolor='k',linewidth=0.8)

    
    ax.coastlines('50m', 'k', linewidth=0.8, zorder=2)
    if Extent !=None:
        extent = Extent
    else:
        extent = [-122., -72, 23., 50.]
    
    ax.set_extent(extent,ccrs.PlateCarree()) 
    
    cmap = refl_cmap
    vmin=-40
    vmax=84.5
    cs = ax.pcolormesh(lon, lat, reflec,cmap=cmap,transform=ccrs.PlateCarree(),vmin=vmin,vmax=vmax)
    cbar = plt.colorbar(cs, orientation='horizontal')
    posn = ax.get_position()
    outline_effect = [patheffects.withStroke(linewidth=3, foreground='k')]
    cbar.ax.set_position([posn.x0+0.001, posn.y0-0.001,
                            (posn.x1-posn.x0)/2, posn.height])
    params = {
              "xtick.color" : "k",
              "ytick.color" : "k",
              "font.size" : 10,
                  }
    #plt.rcParams.update(params)
    cbar.set_ticks([])
    cbar.ax.set_xticklabels([])
    ticks = np.arange(-20,80,10)
    Y = 23
    for count,ele in enumerate(ticks,0): 
        cbar.ax.text(ele, Y, ticks[count], ha='center', va='center',path_effects=outline_effect,color="w",fontsize=6)
    
    #ax.background_patch.set_facecolor('k')
    make_text_time_right(ax,start,title_time)
    make_text_time_left(ax)
    
    #plt.show()
    plt.savefig(f"{save_path}composite_reflectivity_thredds_{file_time}.png",bbox_inches="tight",dpi=200)
    plt.close(fig)

In [29]:
Year = 2020
Month = 4
Day = 29
Hour = 0
Min = 5
start = datetime(Year,Month,Day,Hour,Min)
save_path = "/Users/chowdahead/Desktop/"

In [30]:
%%time
data,times,lon,lat = query_reflectivity_radar_data(start,minute_delta=50,hour_delta=0,day_delta=0,
                                  west=-122, east=-72, south=23, north=50)

query successful!
CPU times: user 3.39 s, sys: 1.62 s, total: 5.02 s
Wall time: 1min 23s


In [31]:
%%time
for i in range(len(times)):
    make_map(data,lon,lat,save_path,time_index=i,Extent=None)

20165.0
2020-04-29 00:05Z


/Users/chowdahead/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:4: RuntimeWarning: invalid value encountered in less
  after removing the cwd from sys.path.


20170.0
2020-04-29 00:10Z
20175.0
2020-04-29 00:15Z
20180.0
2020-04-29 00:20Z
20185.0
2020-04-29 00:25Z
20190.0
2020-04-29 00:30Z
20195.0
2020-04-29 00:35Z
20200.0
2020-04-29 00:40Z
20205.0
2020-04-29 00:45Z
20210.0
2020-04-29 00:50Z
20215.0
2020-04-29 00:55Z
CPU times: user 11min 52s, sys: 46.6 s, total: 12min 38s
Wall time: 18min 43s
